In [ ]:
import cv2 
import csv
import glob
import natsort
import numpy as np
import re
import pickle

In [ ]:
user = 'siddhi'
#user = 'yagnesh'

In [ ]:
def get_numbers_from_filename(filename):
    'utility function to get first number from a file name'
    return re.search(r'\d+', filename).group(0)

In [ ]:
def markSafeDurationForVideo(video_path):

    '''
    Marks safe duration from a video with frame level precision

    Parameters:
    video_path(string) : path of the video for which you want to mark safe duration

    Working:

    1. When the opencv window for video apperars, press 'a' to start the annotations
    2. When the safe duration starts, press 's' to note that particular frame number
    3. When the safe duration ends, press 'e' to note that particular frame number
    4. To pause/play the video, press 'p'
    5. To quit the vide, press 'q'

    Additional note:

    1. You can mark multiple safe durations in a video
    2. You can press '0' if you feel that safe duration had started from the very 1st frame of video but you could not press 's' on time.
    3. If you pressed 's' but the video ends before pressing 'e', last frame_no of the video will be noted as corresponding frame_no safe duration end   
    4. You must press 'e' after 's' before starting to note another safe duration interval

    Returns:
    safe_duration_list(list) : a list of the type [safe_start1, safe_end1, safe_start2, safe_end2,......]
    flag(0/1) : 1 if annotations are proper, 0 if keys are pressed in wrong sequence(e.g., 'e' pressed without corresponding 's')

    '''

    # for resizing a window for video (optional depending on screen size)
    cv2.namedWindow(video_path, cv2.WINDOW_AUTOSIZE)
    #cv2.resizeWindow(video_path, height=760, width=1366)

    # setting font properties for displaying frame number with video
    font = cv2.FONT_HERSHEY_COMPLEX  # fontFace
    fontScale = 1  # fontScale
    fontColor = (255,255,255) # fontColor
    lineType = 2 # lineType
    
    # capture a video from given path
    scanned_video = cv2.VideoCapture(video_path)
    no_frames = int(scanned_video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_count = -1
    flag = 1

    ss = []  # safe duration start list
    se = [] # safe duration end list
    safe_duration_list = [] # final list containing ss and se

    if (scanned_video.isOpened()== False): # check if there is error in opening a video
        raise Exception("check the videopath or error opening video")

    while(scanned_video.isOpened()):

        ret, frame = scanned_video.read() 
        key = cv2.waitKey(34)
        
        if(ret == True):

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(33)
                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)

                    if key_init == ord('a'):  # to start a video when the video is loaded
                        break
            
            frame_count = frame_count + 1
            
            cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
            cv2.imshow(video_path,frame)

            if key == ord('p'): # pause a video

                while True:
                    
                    key2 = cv2.waitKey(33)

                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow(video_path, frame)
                    
                    if key2 == ord('p'): # resume video after pausing
                        break
            
            if key == ord('r'): # key for the case when there is safe duration at the 0th frame(at the very start of the video)

                ss.append(0) # add 0th frame in list of safe duration start
                if (len(ss) != 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(0)
            
            if key == ord('s'): # save safe duration start

                ss.append(frame_count)
                if (len(ss) != len(se) + 1):
                    print("You have made some mistake")
                    flag = 0
                    break
                    
                safe_duration_list.append(frame_count)
                
            if key == ord('e'): # save safe duration end

                se.append(frame_count)
                if (len(se) != len(ss)):
                    print("You have made some mistake")
                    flag = 0
                    break
                safe_duration_list.append(frame_count)
            
            if key == ord('q'): # close video window
                break

        else:
            break


    scanned_video.release()
    cv2.destroyAllWindows()

    # if length of safe duration list is odd length, i.e. one safe duration end is left to be appended,      so add last frame of the video as safe duration end.
    if(len(safe_duration_list) % 2 !=0):
        safe_duration_list.append(no_frames - 1)
    
    return safe_duration_list, flag


In [ ]:

def save_labels_csv(videos_list, csv_file, pickle_list_path):

    '''
    For a given list of video paths, saves the pickle list file and csv file for annotations

    Parameters:
    videos_list : list of video paths for which you want to save the annotations
    csv_file : path of csv file
    pickle_list_path : path of pickle list

    Working:
    After giving the correct input parameters, follow the user friendly on-screen instructions to annotate the videos

    '''
    
    csv_writer = csv.writer(open(csv_file,"w"))
    open_file = open(pickle_list_path, "rb")
    pickle_list = pickle.load(open_file)
    open_file.close()
    
    for video_path in videos_list:
        
        print('\nstarting for', video_path)
        video_no = int(get_numbers_from_filename(video_path)) # function for getting video number for current video
        ready_flag = str.lower(input('type "y" if you are ready to annotate the video and "n" to quit the program\n'))
        if(ready_flag == 'y'):
            correct_flag = 1
            surity_flag = 0
            while(surity_flag == 0 or correct_flag == 0): # loop through the video untill surity and correct flag is not valid
                safe_duration_list, correct_flag = markSafeDurationForVideo(video_path)
                if(correct_flag == 0):
                    print("press keys carefully this time!")
                    input("press any key when you are ready")
                else:
                    surity_flag = int(input('Enter 0 to repeat the annotations process and 1 to proceed to next video\n')) # for assuring that video is annotated as per needed by user
                if(surity_flag == 1 and correct_flag == 1):
                    break

            #csv_writer.writerow(safe_duration_list)
            pickle_list[video_no-1] = safe_duration_list # add list to pickle file with video number as index

            print("safe_duration_list", safe_duration_list)
            print('------------------------------------------')
            
        else:
            
            print('\nTake a break! You must be tired')
            break
            
    open_file = open(pickle_list_path, "wb")
    pickle.dump(pickle_list, open_file)
    open_file.close()
    csv_writer.writerows(pickle_list)
    print("list and csv file saved")

In [ ]:


if(user == 'siddhi'):
    video_folder = 'C:/RoadCrossingAssistant/Data/videos'
    videos_list = glob.glob(video_folder + '/video*.MOV')
    videos_list = (natsort.natsorted(videos_list))
    pickle_list_path = 'C:/RoadCrossingAssistant/Data/labels_framewise_list.pkl'
    csv_file_path = 'C:/RoadCrossingAssistant/Data/labels_framewise_csv.csv'

elif(user == 'yagnesh'):

    video_folder = ''
    videos_list = glob.glob(video_folder + '/video*.MOV')
    videos_list = (natsort.natsorted(video_list))
    pickle_list_path = ''
    csv_file_path = ''

In [ ]:
save_labels_csv(videos_list, csv_file_path, pickle_list_path)

In [ ]:
#labels_framewise = [[] for i in range(104)]
# open_file = open(file_name, "wb")
# pickle.dump(my_list, open_file)
# open_file.close()

open_file = open(pickle_list_path, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

print(len(loaded_list),'\n', loaded_list)
print(loaded_list[np.array([0,1,5,7,9])])

In [ ]:
def visualize_annotations(video, safe_duration_list):

    '''
    Function to visualize the annotaions you did

    Paramters:
    video(string) : path of video
    safe_duration_list : safe_duration_list that is returned by the function markSafeDurationForVideo()

    Working:

    When the opencv window for video apperars, press 'a' to start

    '''


    cap = cv2.VideoCapture(video)
    cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(video, height=650, width=1156)

    no_safe_durations = int(len(safe_duration_list)/2)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    labels = [0]*no_frames
    for i in range(no_safe_durations):
        ss = safe_duration_list[i*2]
        se = safe_duration_list[i*2 + 1]
        labels[ss:se+1] = [1]*(se+1-ss)
    
    frame_count = -1
    


    while(cap.isOpened()):

        success,frame = cap.read()
        #print(i)

        if success == True:

            if(frame_count == -1):
                while True:

                    key_init = cv2.waitKey(25)
                    cv2.imshow(video, frame)

                    if key_init == ord('a'):
                        break

            frame_count = frame_count + 1
            

            if(labels[frame_count] == 1):

                cv2.rectangle(frame, (1576,3), (1890,95), (0,200,0), thickness=-1)
                frame_save = cv2.putText(frame, 'SAFE', (1580,90), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,255,255), 6, cv2.LINE_AA)
                cv2.imshow(video, frame_save)
                cv2.waitKey(34)

            else:

                cv2.rectangle(frame, (1396,3), (1890,95), (0,0,200), thickness=-1)
                frame_save = cv2.putText(frame, 'UNSAFE', (1400,90), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,255,255), 6, cv2.LINE_AA)
                cv2.imshow(video, frame_save)
                cv2.waitKey(34)

        else:
            break

    
    cap.release()

    cv2.destroyAllWindows()

#visualize_annotations("/home1/RoadCrossingAssistant_FY_Project_Data/videos/video2.MOV", [52,143])



In [ ]:
def verify_annotations(pickle_list, videos_list):

    '''
    For a given list of video paths and the saved list for annotations, visualize annotaions one by one
    Parameters:
    videos_list : list of video paths for which you want to save the annotations
    pickle_list : path of saved pickle list
    '''

    open_file = open(pickle_list, "rb")
    labels_framewise = (pickle.load(open_file))[10:]
    open_file.close()

    for i in range(len(videos_list)):

        safe_duration_list = labels_framewise[i]
        video = videos_list[i]

        print('\nstarting for', video)
        ready_flag = str.lower(input('type y if you are ready to start the video and n to quit the program\n'))


        if(ready_flag == 'y'):

            surity_flag = 0
            while surity_flag == 0:
                visualize_annotations(video, safe_duration_list)
                surity_flag = int(input('type 1 if you are sure and 0 to replay the video\n'))

        else:

            print('\nTake a break! You must be tired')
            break

In [ ]:
verify_annotations(pickle_list_path, videos_list[10:])